In [ ]:
# python3 -m venv elbaff_iesta_venv --prompt="elbaff_iesta_venv"
# source elbaff_iesta_venv/bin/activate

In [1]:
from nlpaf.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np

import iesta.properties as prop  
from iesta.machine_learning.dataloader import IESTAData, METHODOLOGY


/home/elba_ro/repos/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/elba_ro/repos/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:

liberal_each_dataloader = IESTAData(ideology=prop.LIBERAL_IDEOLOGY, methodology=METHODOLOGY.EACH)
liberal_training_each_df, liberal_data_path = liberal_each_dataloader.get_training_data()
liberal_each_dataloader.pivot_df


/home/elba_ro/repos/conf22-style-transfer/iesta/../data/splitted_liberal_debate_arguments_effect_test0.3_random2.parquet


effect,effective,ineffective,okay,provocative
split,,,,
test,248,5349,969,92
training,1722,37713,6835,522
validation,613,10749,1828,204


In [3]:
liberal_each_dataloader.pivot_binary_effect

binary_effect,effective,ineffective
split,,
test,248,6410
training,1722,45070
validation,613,12781


In [4]:
liberal_training_each_df.columns.tolist()

['id',
 'debate_id',
 'p_name',
 'effects',
 'effect_count',
 'top_effect',
 'effect',
 'category',
 'round',
 'argument',
 'split',
 'cleaned_text',
 'binary_effect']

In [5]:
class TransformerBasedFeaturePipeline(Pipeline):
    

    def __init__(self,
                 input= None,
                 load_default_pipe_configs = True,
                 extended_pipe_configs:dict = None,
                 save_output= False,
                 out_path = None,
                 argument_col:str = "cleaned_text"
                 ):
        super().__init__(input,
                 load_default_pipe_configs,
                 extended_pipe_configs,
                 save_output,
                 out_path)
        self.argument_col= argument_col
        
    def process_input(self) -> list:
        processed = []
        txt_df = self.input[["id", self.argument_col]].copy()
        txt_df =txt_df.rename(columns={"id": "input_id", self.argument_col: "text"},)
        
        for idx, row in txt_df.iterrows():
            processed.append((row.text, {"input_id": row.input_id}))

        return processed
    
    
    def init_and_run(self):
        self.add_annotation_pipe(name = "sentencizer", save_output= False,is_spacy=True, is_native=True)
        self.add_annotation_pipe(name = "EmotionPipeOrchestrator", save_output= True, is_spacy=True)
        #self.add_annotation_pipe(name = "HedgePipeOrchestrator",   save_output= True, is_spacy=True)
        self.add_annotation_pipe(name = "ToxicityOrchestrator",    save_output= True, is_spacy=True)
        self.init_pipe_stack()
        

In [7]:
from nlpaf.util.timer import Timer
from tqdm.auto import tqdm
import pyarrow.parquet as pq
from pathlib import Path
import math
from iesta import utils

batch_size = 100
counter = 1


total = math.ceil(len(liberal_training_each_df)/(batch_size))

parquet_file = pq.ParquetFile(liberal_data_path)
pipeline = TransformerBasedFeaturePipeline(save_output= True)
pipeline.init_and_run()
tqdm.pandas()


out_path = f"../data/extracted_features/liberal_emotion-toxic_{batch_size}"
utils.create_folder(out_path)
out_file = out_path+"/liberal_batch{}_{}_emotion-toxic.parquet"


tqdm.pandas()

for batch in tqdm(parquet_file.iter_batches(batch_size=batch_size), total=total ):
    file = out_file.format(batch_size, counter, batch_size)
    if  Path(file).is_file():
        counter = counter+1
        continue
    batch_df = batch.to_pandas()
        
    ## RESET
    pipeline.reset_input_output()
    pipeline.out_path = file
    
    pipeline.set_input(batch_df)
    t = Timer(name=f"chunck_{counter}")
    t.start()
    
    pipeline.annotate()
    pipeline.save()
    t.stop()
    #pipeline.out_df.head()
    counter = counter+1
    #break


nlpaf        INFO     adding pipe with name EmotionPipeOrchestrator
nlpaf        INFO     orchestrator was initialized successfully
nlpaf        INFO     adding pipe with code emotion_hartmann_component
nlpaf        INFO     adding pipe with name ToxicityOrchestrator
nlpaf        INFO     orchestrator was initialized successfully
nlpaf        INFO     adding pipe with code toxicity_component
nlpaf        INFO     Defining pipe default and spacy stacks
Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification 

In [ ]:
lib_transformers_df = pd.read_parquet("../data/extracted_features/features_liberal_training_each.parquet")

lib_transformers_df.head()
lib_transformers_df['hedge_dominant'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['emotion_hartmann_label'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['toxicity_dominant'].value_counts().to_frame().plot(kind="bar")

lib_transformers_df[["emotion_hartmann_anger",
                     "emotion_hartmann_disgust",
                    "emotion_hartmann_fear", 
                    "emotion_hartmann_joy",
                    "emotion_hartmann_neutral",
                    "emotion_hartmann_surprise",
                    "emotion_hartmann_sadness"]].plot.hist(bins=12, alpha=0.5)